In [1]:
# Author: Joshua Gregory
# Date: Sept. 2025
# Descr: Uses BFS & Iterative DFS to attempt to find path to goal state in 8-Puzzle Problem variant.

In [2]:
from collections import deque
import random

In [3]:
# Global fields
GOAL_STATE = [[1,2,3], [8,0,4], [7,6,5]]  # solution state as 2D array
GRID_SIZE = 3
row_moves = [0, 0, -1, 1]  # L/R/U/D directional values for rows  *Source: GeeksForGeeks/Stack Overflow
col_moves = [-1, 1, 0, 0]  # L/R/U/D directional values for cols  *Source: GeeksForGeeks/Stack Overflow

In [4]:
# Puzzle state space 2d arr = [[0,1,2], [3,4,5], [6,7,8]]
class Eight_Puzzle:
  def __init__(self, grid_state, blank_x, blank_y, depth_level):
    """
    Eight_Puzzle constructor to define & initialize object fields.
    @param grid_state: attribute for 2d array (grid) puzzle state.
    @param blank_x: attribute for row position of blank tile.
    @param blank_y: attribute for column position of blank tile.
    @param depth_level: attribute for depth of current state node.
    """
    self.grid_state = grid_state    # puzzle state 2d array grid
    self.blank_x = blank_x          # row coord of blank tile (value 0)
    self.blank_y = blank_y          # col coord of blank tile (value 0)
    self.depth_level = depth_level  # depth relative to root

In [5]:
# Generate root state space
def get_root_grid():
  """
  Creates grid for 8-puzzle root state as a 2d array with elements [0,8] shuffled.
  @return: Root grid for 8-puzzle, row & column coords of blank tile.
  """
  tile_values = list(range(9))  # list of integers [0,8], 0 = blank tile
  random.shuffle(tile_values)   # shuffle tile positions

  root_grid = [tile_values[i : i+3] for i in range(0, 9, GRID_SIZE)]  # convert to 3x3 grid

  for blank_x in range(GRID_SIZE):
    for blank_y in range(GRID_SIZE):          # search each col in each row
      if root_grid[blank_x][blank_y] == 0:    # for blank tile with value 0
        return root_grid, blank_x, blank_y    # return grid & xy coords of blank tile

In [6]:
# Print grid state to console
def print_to_grid(grid_state):
  """
  Prints current grid state as a grid with underscore for blank tile.
  @param grid_state: Current grid state represented as a 2d array.
  """
  print("--------")
  for row in grid_state:
    print(' '.join('_' if val == 0 else str(val) for val in row))
  print("--------")

In [7]:
# Check if goal state reached
def state_equals_goal(grid_state):
  """
  Checks if input 2d array (grid state) matches GOAL_STATE.
  @param grid_state: 2d array (grid) representing puzzle state.
  @return: (Boolean) T if all elements in input 2d array match GOAL_STATE, else F.
  """
  return grid_state == GOAL_STATE

In [8]:
# Validate move is legal
def adj_in_range(adj_x, adj_y):
  """
  Validates that a tile position is in grid state range.
  @param adj_x: Row position of target adjacent tile.
  @param adj_y: Column position of target adjacent tile.
  @return: (Boolean) T if tile is in grid state range, else F.
  """
  return 0 <= adj_x < GRID_SIZE and 0 <= adj_y < GRID_SIZE

In [9]:
# Swap blank tile using row_moves/col_moves
def move_blank_tile(grid_state, blank_x, blank_y, adj_x, adj_y):
  """
  Updates grid state via swapping blank tile & adjacent tile values.
  @param grid_state: Current state of 8-puzzle grid.
  @param blank_x: Row position of current blank tile (value 0).
  @param blank_y: Column position of current blank tile (value 0).
  @param adj_x: Row position of target adjacent tile.
  @param adj_y: Column position of target adjacent tile.
  @return: Updated state with swapped tiles.
  """
  new_state = [row[:] for row in grid_state]  # copy input grid state

  temp = new_state[blank_x][blank_y]                     # store in temp var
  new_state[blank_x][blank_y] = new_state[adj_x][adj_y]  # swap with adj tile
  new_state[adj_x][adj_y] = temp                         # swap with blank tile

  return new_state  # return updated state

In [16]:
def run_bfs(grid_state, blank_x, blank_y):
  """
  Searches tiers completely before visiting children nodes.
  @param grid_state: Puzzle state as 2d array via get_root_grid().
  @param blank_x: Row position of blank tile (value 0).
  @param blank_y: Column position of blank tile (value 0).
  """
  Q = deque()   # FIFO queue for searching complete tiers
  SEEN = set()  # re-init per instance, prevents cycling

  Q.append(Eight_Puzzle(grid_state, blank_x, blank_y, 0))  # create object & enqueue
  SEEN.add(tuple(map(tuple, grid_state)))  # 2d array state = immutable tuple of tupled values ((r1),(r2),(r3))

  while Q:                  # while states remain in queue
    current = Q.popleft()   # pop & store first (leftmost) node

    print(f"\nDepth = {current.depth_level}:")  # print current depth
    print_to_grid(current.grid_state)           # print current state as NxN grid

    if state_equals_goal(current.grid_state):
      print(f"\n~ Goal state reached at depth {current.depth_level} via BFS! ~")  # solution state reached
      return  # exit

    for i in range(4):  # loop row_moves & col_moves while i = [0,3]
      adj_x = current.blank_x + row_moves[i]  # get row coords for possible blank tile position
      adj_y = current.blank_y + col_moves[i]  # get col coords

      if adj_in_range(adj_x, adj_y):  # validate each possible xy in range
        new_state = move_blank_tile(current.grid_state, current.blank_x, current.blank_y, adj_x, adj_y)  # move blank tile

        state_tuple = tuple(map(tuple, new_state))  # 2d array = hashable tuple of tuples

        if state_tuple not in SEEN:
          SEEN.add(state_tuple)   # add new state tuple to seen nodes set
          Q.append(Eight_Puzzle(new_state, adj_x, adj_y, current.depth_level+1))  # create new object & enqueue

  print("~ Goal state unreachable from root state via BFS. ~")  # all possible moves exhausted

In [15]:
# DFS
def run_idfs(grid_state, blank_x, blank_y):
  """
  Searches branches completely before backtracking & visiting sibling nodes.
  @param grid_state: Current state of 8-puzzle grid.
  @param blank_x: Row position of blank tile (value 0).
  @param blank_y: Column position of blank tile (value 0).
  """
  S = []        # LIFO stack for exploring branch depths (iteratively deepening)
  SEEN = set()  # re-init per instance, stores seen states to prevent cycling

  S.append(Eight_Puzzle(grid_state, blank_x, blank_y, 0))  # create object & push to stack
  SEEN.add(tuple(map(tuple, grid_state)))  # convert to hashable tuple, add to seen nodes set

  while S:             # while nodes (states) remain on stack
    current = S.pop()  # pop the most recently added node (state)

    print(f"\nDepth = {current.depth_level}:")  # print current depth
    print_to_grid(current.grid_state)           # print state as NxN grid

    if state_equals_goal(current.grid_state):
      print(f"\n~ Goal state reached at depth {current.depth_level} via IDFS! ~")  # solution state reached
      return  # exit

    for i in range(4):  # loop through row_moves & col_moves
      adj_x = current.blank_x + row_moves[i]  # new x = current + L/R/U/D
      adj_y = current.blank_y + col_moves[i]  # new y = current + L/R/U/D

      if adj_in_range(adj_x, adj_y):  # validate adj xy is in grid range & swap tiles
        new_state = move_blank_tile(current.grid_state, current.blank_x, current.blank_y, adj_x, adj_y)

        state_tuple = tuple(map(tuple, new_state))  # convert new state to hashable tuple

        if state_tuple not in SEEN:
          SEEN.add(state_tuple)   # add state tuple to seen nodes set
          S.append(Eight_Puzzle(new_state, adj_x, adj_y, current.depth_level+1))  # add stack with new blank tile xy & incremented depth level

  print("\n~ Goal state unreachable from root state via IDFS. ~")  # all possible moves exhausted

In [12]:
# Helper method to display menu & return validated user input
def get_menu_choice():
  while True:  # loop menu prompt until valid input
    print("\n+------------------------------+")
    print("| 8-PUZZLE BFS/DFS SOLVER MENU |")
    print("+------------------------------+")
    print("| 1. New 8-Puzzle Grid         |")
    print("| 2. Run BFS Path Finder       |")
    print("| 3. Run IDFS Path Finder      |")
    print("| 4. Exit program              |")
    print("+------------------------------+")

    choice = input("\nEnter a menu option: ").strip()  # save input without spaces

    if choice == '1' or choice == '2' or choice == '3' or choice == '4':
      return choice
    else:
      print("ERROR: Invalid input (enter menu option 1, 2, 3, or 4).")

In [17]:
# Driver UI
def main():
  root_puzzle = None  # initialize puzzle object to null to check for

  while True:
    choice = get_menu_choice()  # menu prompt for validated input

    # MENU 1: GENERATE NEW 8-PUZZLE GRID
    if choice == '1':
      print("Generating new 8-puzzle grid...")
      root_puzzle, root_x, root_y = get_root_grid()  # get grid & xy coords of value 0

      print("\nNew 8-Puzzle root state:")
      print_to_grid(root_puzzle)  # print initial puzzle state

    # MENU 2: SOLVE VIA BFS
    if choice == '2':
      if not root_puzzle:  # if puzzle not created first
        print("ERROR: Requires an 8-Puzzle grid (use menu option 1).")
      else:
        print()  # formatting
        run_bfs(root_puzzle, root_x, root_y)  # input = returns from get_root_grid()

    # Menu 3: Solve via DFS
    if choice == '3':
      if not root_puzzle:  # if new puzzle not created
        print("ERROR: Requires an 8-Puzzle grid (use menu option 1).")
      else:
        print()  # formatting
        run_idfs(root_puzzle, root_x, root_y)  # input = returns from get_root_grid()

    # Menu 4: Exit
    if choice == '4':
      print("Exiting program... Goodbye!")
      return

In [ ]:
# Tester
if __name__ == '__main__':
  main()


+------------------------------+
| 8-PUZZLE BFS/DFS SOLVER MENU |
+------------------------------+
| 1. New 8-Puzzle Grid         |
| 2. Run BFS Path Finder       |
| 3. Run IDFS Path Finder      |
| 4. Exit program              |
+------------------------------+

Enter a menu option: 1
Generating new 8-puzzle grid...

New 8-Puzzle root state:
--------
_ 2 3
1 7 6
8 5 4
--------

+------------------------------+
| 8-PUZZLE BFS/DFS SOLVER MENU |
+------------------------------+
| 1. New 8-Puzzle Grid         |
| 2. Run BFS Path Finder       |
| 3. Run IDFS Path Finder      |
| 4. Exit program              |
+------------------------------+
